In [2]:
import math
import pandas as pd
import numpy as np
from math import sqrt

In [10]:
data = pd.read_csv("clean_median_metrix_data.csv")
data = data.dropna()
data['spread'] = data['spread']/1000
data['target_median'] = data['target_median']/1000
data.head()

,spread,target_median,number_KNN,ABS Metrics,fsignal
1,1.000,1.000,1.0,0.00,0.0
3,1.393,1.393,1.0,0.00,0.0
5,1.610,1.824,2.0,0.12,0.0
7,1.018,1.018,1.0,0.00,0.0
9,2.517,1.345,4.0,0.87,0.0


In [12]:
feature_matrix = data.values[:,:-1]
weights = [1,2,3,4]

In [36]:
obs = data.values[:,-1]
step_size = 10-6

In [16]:
scores =  get_score(weights,feature_matrix)

In [29]:
calculate_log_likelihood(feature_matrix,obs,weights)

-24537.472194654125

In [52]:
get_logistic_regression(feature_matrix, weights, obs,step_size)

[111460887040859.08, 112207350253620.44, 17119976097505.06, 322992202096.92664]

In [15]:
def get_score(weights,feature_matrix):
    scores = np.dot(feature_matrix, weights)
    return scores

In [19]:
# calculate probability 
def calculate_prob(scores):
    prob_pred = []
    for score in scores:
        prob = 1/(1+math.exp(-score))
        prob_pred.append(prob)
    return prob_pred

In [21]:
# get accuracy 
def get_classification_accuracy(pred, obs):
    num_correct = sum(pred==obs)
    accuracy = num_correct/len(pred)
    return accuracy

In [28]:
# compute derivative of log likelihood with respect to a single coefficient 
def calculate_log_likelihood(feature_matrix,obs,weights):
    pred = np.dot(feature_matrix, weights)
    logexp = 1/(1+np.exp(-pred))
    lp = np.sum((obs -1)*pred - logexp)
    return lp

In [42]:
# logistic regression -- gradient descent 
# 1. gradient function 
# 2.logistic regression 
def get_gradient(error,feature_matrix):
    derivatives = np.dot(error,feature_matrix)
    return derivatives 

def get_logistic_regression(feature_matrix, weights, obs,step_size):
    D = feature_matrix.shape[1]
    pred = np.dot(feature_matrix, weights)
    error = obs - pred
    for j in range(D):
        derivatives = get_gradient(error,feature_matrix[:,j])
        weights[j] += step_size * derivatives
    return weights

In [53]:
# build decision tree
# 1. count number of mistake while predicting marjority class 
def intermediate_node_num_mistake(labels_in_node):
    if (len(labels_in_node) == 0):
        return 0
    num_positive = (labels_in_node == 1).sum()
    num_negative = (labels_in_node == -1).sum()
    return num_negative if num_positive > num_negative else num_positive

In [55]:
##2. pick up best feature to split 
def best_spliting_feature(data,features, target):
    best_feature = []
    best_error = 10
    num_record = len(data)
    
    for feature in features:
        right_size = data[data[feature] == 1]
        left_side = data[data[feature] == 0]
        left_mistake  =  intermediate_node_num_mistake(left_side)
        right_mistake = intermediate_node_num_mistake(right_side)
        avg_mistake = (left_mistake+ right_mistake)/num_record
        if avg_mistake< best_error:
            best_error = avg_mistake
            best_feature = feature 
    return best_feature

In [56]:
# 3. create leaf 
def create_leaf(target_value):
    leaf = {"spliting feature": None,
           "left" : None,
            "right" : None,
            "is_leaf":True
           }
    num_ones = len(target_values[target_values == 1])
    num_minus_ones = len(target_values[target_value == -1])
    
    if num_ones> num_minus_one:
        leaf['prediction'] = 1
    else:
        leaf['prediction'] = -1
    return leaf 

In [ ]:
## stoping condition 1: all points in a node are in the same class 
## stoping condition 2: no  more feature 
## stoping condition 3: max depth 
def decision_tre_create(data, features,'safe_loan',max_depth = 3):
    remaining_feature = feature[:]
    target_values = data[target]
    print('--------------------------------------------------------------------------')
    print("subtree , depth = %s(%s data pints).") %(current_depth, len(target_values))
    
    if intermediate_node_num_mistake(labels_in_node) == 0:
        print("stop condition 1 reached")
        return create_leaf(target_values)
    
    if remaining_feature == []:
        print("stop condiction 2 reached")
        return create_leaf(target_values)
    if current_depth >= max_depth:
        print("reached maximum depth. stop for now")
        return create(targe_values)
    ## fine the best splitting feature 
    splitting_feature = best_spliting_feature(data,features, target)
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    remaining_feature.remove(spliting_feature)
    print("split on feature %s.(%s ,%s)"%(splitting_feature,len(left_split),len(right_split)))
    
    if len(left_split) == len(data):
        print("create leaf node")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print("create leaf node")
        return create_leaf(right_split[target])
    
    # repeat on left and right subtree
    left_tree = decision_tree_create
    right_tree = decicsion_tree_create 
    return {"is_leaf"          : False,
            "predction"        : None,
            "spllting_feature" : splitting_feature,
            "left"             : left_tree, 
            "right"            :right_tree
           }

In [57]:
def cout_node(tree):
    if tree["is_leaf"]:
        return 1
    return 1 + count_nodes(tree['left']) + count_node(tree['right'])

In [1]:
## explore loan data 

In [142]:
loans = pd.read_csv("decision_tree_data.csv")
loans = loans.iloc[:,:]
loans.head()

C:\Users\AQi\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,disbursement_method,debt_settlement_flag
0,8000,8000,8000.0,36 months,10.99%,261.88,B,B2,PARTS MANAGER,2 years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Cash,N
1,12000,12000,12000.0,36 months,7.62%,373.94,A,A3,Systems Engineer,3 years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Cash,N
2,20800,20800,20800.0,36 months,13.53%,706.16,B,B5,Operations Manager,10+ years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Cash,N
3,10000,10000,10000.0,36 months,9.67%,321.13,B,B1,Registered Nurse,7 years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Cash,N
4,24000,24000,24000.0,36 months,13.53%,814.80,B,B5,driver,10+ years,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Cash,N


In [143]:
features = ['grade','term','emp_length',"home_ownership"]
target = ['debt_settlement_flag']
features + target

['grade', 'term', 'emp_length', 'home_ownership', 'debt_settlement_flag']

In [139]:
loans = loans[features + target]

In [144]:
safe_loan_raw = loans[loans[target] == "Y"]
risky_loan_raw = loans[loans[target] == "N"]
percentage = len(safe_loan_raw)/len(risky_loan_raw)
print(percentage)

1.0
